In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")


In [5]:
# reflect an existing database into a new model

Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found

print(Base.classes.keys())


['measurement', 'station']


In [7]:
# Save references to each table

measurement = Base.classes.measurement
station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB

session = Session(engine)


# Exploratory Precipitation Analysis

In [14]:
# Inspect the 'measurement' file to locate the actual name of the date column.

inspector = inspect(engine)
inspector.get_columns("measurement")

[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'station',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'date',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'prcp',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'tobs',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

In [49]:
# Find the most recent date in the data set.
sesson = Session(engine)

session.query(measurement.date).\
              group_by(measurement.date).distinct().\
              order_by(measurement.date.desc()).all()
        

[('2017-08-23'),
 ('2017-08-22'),
 ('2017-08-21'),
 ('2017-08-20'),
 ('2017-08-19'),
 ('2017-08-18'),
 ('2017-08-17'),
 ('2017-08-16'),
 ('2017-08-15'),
 ('2017-08-14'),
 ('2017-08-13'),
 ('2017-08-12'),
 ('2017-08-11'),
 ('2017-08-10'),
 ('2017-08-09'),
 ('2017-08-08'),
 ('2017-08-07'),
 ('2017-08-06'),
 ('2017-08-05'),
 ('2017-08-04'),
 ('2017-08-03'),
 ('2017-08-02'),
 ('2017-08-01'),
 ('2017-07-31'),
 ('2017-07-30'),
 ('2017-07-29'),
 ('2017-07-28'),
 ('2017-07-27'),
 ('2017-07-26'),
 ('2017-07-25'),
 ('2017-07-24'),
 ('2017-07-23'),
 ('2017-07-22'),
 ('2017-07-21'),
 ('2017-07-20'),
 ('2017-07-19'),
 ('2017-07-18'),
 ('2017-07-17'),
 ('2017-07-16'),
 ('2017-07-15'),
 ('2017-07-14'),
 ('2017-07-13'),
 ('2017-07-12'),
 ('2017-07-11'),
 ('2017-07-10'),
 ('2017-07-09'),
 ('2017-07-08'),
 ('2017-07-07'),
 ('2017-07-06'),
 ('2017-07-05'),
 ('2017-07-04'),
 ('2017-07-03'),
 ('2017-07-02'),
 ('2017-07-01'),
 ('2017-06-30'),
 ('2017-06-29'),
 ('2017-06-28'),
 ('2017-06-27'),
 ('2017-06-26'

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()